In [45]:
import requests
from bs4 import BeautifulSoup 
import random
import pandas as pd
import json
import time
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [46]:
def get_imag(url,save_path):
    browser = selenium.webdriver.Chrome() # 打开谷歌浏览器
    browser.get(url)
    for i in range(1,20):
        time.sleep(0.2)#稍微停顿
        browser.execute_script("window.scrollBy(0,2000)")        
    node = browser.find_elements_by_xpath("//div[@id = 'J-detail-content']//img") # 寻找节点
    j=0
    if node!=[] :
        print(1)
        for i in node:
            j=j+1#备用图片名
            with open(save_path.rstrip('/')+'/'+i.get_attribute('src')[-5:],'wb') as f:
                img_url=i.get_attribute('src')
                print(img_url)
                headers={
                    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'
                }
                img_get=requests.get(img_url,headers=headers).content
                f.write(img_get)
    else:
        print(2)
        node = browser.find_elements_by_xpath("//div[@id = 'J-detail-content']/div/div")
        id_list=[]#图片id名
        for i in node:
            id_list.append(i.get_attribute('data-id'))
        style_inf=browser.find_element_by_xpath("//div[@id = 'J-detail-content']//style")
        style=style_inf.get_attribute('innerHTML')
        style=str(style)
        while(style.find('background-image:url(')!=-1):
            start=(style.find('background-image:url(')+len('background-image:url('))
            style=style[start:]
            end=style.find(');')
            img_url=style[:end]
            img_url='https:'+img_url
            with open(save_path.rstrip('/')+'/'+img_url[-10:],'wb') as f:
                headers={
                    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'
                }
                img_get=requests.get(img_url,headers=headers).content
                f.write(img_get)
    browser.quit()
    print('Finish!')

In [43]:
def get_all_image(ans,page):
    Headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55",
        "cookie": "__jdv=122270672|direct|-|none|-|1647493502013; __jdu=164749350201356425815; shshshfpa=378323e4-cf10-6ab8-6f67-85eef54ba88b-1647493502; shshshfpb=w5GiBgnUHADfTmx7Wqeau-g; shshshfp=422ab1959481b6e38d8604bd9328dac8; ip_cityCode=549; areaId=7; ipLoc-djd=7-549-558-34702; __jda=122270672.164749350201356425815.1647493502.1648606921.1648646379.4; __jdc=122270672; token=918c1e81d67b5ab3bffc32e8bfe81751,2,915915; __tk=a099cac6b2821d27822c62eda2051af9,2,915915; __jdb=122270672.4.164749350201356425815|4.1648646379; shshshsID=7cf398889fcc32804818069ed03e31ec_4_1648647771748; 3AB9D23F7A4B3C9B=WAHUHBX3N5DU25OGXJGSSITEBFRQSR67LQEBLPZPSFMJTI5D3SWLLRKOQAUZNGAJKEFGYMKO4DLNMQC352FH22VDYU"
    }
    lst_products = []
    allimg_url={}#存放全部图书图片的url
    for pp in range(1,page+1):
        url = 'https://search.jd.com/Search?keyword={}&page={}'.format(ans,pp)
        print(url)
        res = requests.get(url, headers=Headers)# requests.post(url, headers=Headers)
        res.encoding = res.apparent_encoding
        html = res.text
        #DOM
        soup = BeautifulSoup(html,'html5lib')
        lst_li = soup.find('div',id="J_goodsList").find_all('li')
        for idx, li in enumerate(lst_li):
            #time.sleep(10)#睡眠一秒钟
            print(idx)
#             if idx in [0]:
#                 continue;
            product = []
            img_url=[]
            try:
                product_id = li['data-sku']
                product_price = li.find('div',class_="p-price").strong.i.string.strip()
                product_name =  "".join(li.find('div',class_= "p-name").a.em.strings)
                product.append(product_id)
                product.append(product_price)
                product.append(product_name)
                allimg_url[product_name]=[]
            except:
                print("Error in", li )

            try:
                url_cm = 'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId={}&score=0&sortType=5&page=1&pageSize=10&isShadowSku=0&rid=0&fold=1'.format(product_id)
                res = requests.get(url_cm,headers=Headers)
                jd = json.loads(res.text.lstrip('fetchJSON_comment98vv12345(').rstrip(');'))
                data_list = jd['comments']
                comments = " ".join([data['content'] for data in data_list])
                product.append(comments)
                    ##这里是保存图片的url
                for i in range(len(jd['comments'])):
                    if 'images' in jd['comments'][i].keys():
                        for j in range(len(jd['comments'][i]['images'])):
                            allimg_url[product_name].append(jd['comments'][i]['images'][j]['imgUrl'])
                            img_url.append(jd['comments'][i]['images'][j]['imgUrl'])
                #爬取完一本图书，开始进行图片的保存,创建以该图书为名称的文件夹，下设买家秀和卖家秀文件夹
                path='D:\img_save'
                if not os.path.exists(path):
                    os.mkdir(path)
                if os.path.exists(path):
                    os.makedirs(path+'\\'+product_name)
                    os.makedirs(path+'\\'+product_name+'\\'+'卖家秀')
                    os.makedirs(path+'\\'+product_name+'\\'+'买家秀')
            #保存卖家秀的图片
                sell_url="https://item.jd.com/{}.html#none".format(product_id)
                print(sell_url)
                savepath=path+'\\'+product_name+'\\'+'卖家秀'
                print(savepath)
                get_imag(sell_url,savepath)
            #保存买家秀的图片
                for i in range(len(img_url)):
                    picName = img_url[i].split('/')[-1]#图片的名称
                    response = requests.get('http:'+img_url[i],headers=Headers)
                    with open(path+'\\'+product_name+'\\'+'买家秀'+'\\'+picName,'wb') as f:
                        f.write(response.content)
            except:
                print("Error in Comments", product_id)
            lst_products.append(product)
    print(lst_products)
    pd.DataFrame.from_dict(allimg_url, orient='index').to_csv('JD_product_imgurl.csv',encoding='utf-8-sig')
    pd.DataFrame(lst_products,columns =['product_id','product_id_price','product_id_name','product_id_comments']).to_csv('JD_product_ids.csv',encoding='utf-8-sig')


In [44]:
get_all_image('python',1) #第一个参数是要查找的内容，第二个参数是要从头开始查找的页数

https://search.jd.com/Search?keyword=python&page=1
0
https://item.jd.com/11993134.html#none
D:\img_save\Python编程 从入门到实践 第2版（图灵出品）\卖家秀


E:\Temp/ipykernel_14340/1083889334.py:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  node = browser.find_elements_by_xpath("//div[@id = 'J-detail-content']//img") # 寻找节点


1
https://img30.360buyimg.com/vc/jfs/t27199/364/2343978207/110813/41e64e8d/5bff723dN829c479c.jpg
https://img30.360buyimg.com/vc/jfs/t1/85183/27/22408/195783/6220e1f2E90f673de/dace7c922f1922be.jpg
https://img30.360buyimg.com/vc/jfs/t1/200419/14/14674/615516/6178c11eEfc126148/eeb610b28aedac63.jpg
Finish!
https://search.jd.com/Search?keyword=python&page=2
0
https://item.jd.com/10043837952779.html#none
D:\img_save\京东云&千锋教育联合共创 Python教程自学全套视频教程基础入门人工智能AI爬虫课 Python全栈开发+人工智能+数据分析+就业推荐\卖家秀
1
https://img10.360buyimg.com/imgzone/jfs/t1/212072/2/11393/1079827/61ea2050E60c6d2be/8f841d29bb8e4308.jpg
https://img10.360buyimg.com/imgzone/jfs/t1/90130/17/21707/459232/61ea2050E10357c98/4503e3cc786b0efc.jpg
https://img10.360buyimg.com/imgzone/jfs/t1/142279/11/26863/359835/61ea2050Ea8d07944/bda4439f5c4cdea5.jpg
https://img10.360buyimg.com/imgzone/jfs/t1/104338/8/22437/1187674/61ea2051E95dc4bce/15c3394fbcd1ee39.jpg
https://img10.360buyimg.com/imgzone/jfs/t1/98240/3/22724/328375/61ea2050E0dcf7dd8/288f5be381